In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from tpot.export_utils import set_param_recursive
import xarray as xr
from SALib.sample import saltelli
from SALib.analyze import sobol
import joblib
import re
import os
import dask
import dask.bag as db

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'axes.grid': False,
    'savefig.dpi': 700,
    'font.size': 12,
    'text.usetex': False,
    'figure.figsize': [5, 5],
    'font.family': 'serif',
}
matplotlib.rcParams.update(params)

In [ ]:
from dask_jobqueue import SGECluster
from dask.distributed import Client
cluster = SGECluster()
client = Client(cluster)

In [ ]:
cluster

In [ ]:
client

In [ ]:
cluster.scale(jobs=150)
#cluster.adapt(minimum_jobs=0, maximum_jobs=150)

In [ ]:
client.close()
cluster.close()

In [ ]:
# emulator setup
path = '/nobackup/earlacoa/machinelearning/data/'

with open(path + 'dict_train.pickle', 'rb') as ds:
    dict_train = pickle.load(ds)
    
with open(path + 'dict_test.pickle', 'rb') as ds:
    dict_test = pickle.load(ds)
    
df_train = pd.concat(dict_train, ignore_index=True)
df_test = pd.concat(dict_test, ignore_index=True)

inputs_train = pd.read_csv(path + 'latin_hypercube_inputs_train.csv')
inputs_test = pd.read_csv(path + 'latin_hypercube_inputs_test.csv')
X_train, X_test = inputs_train.values, inputs_test.values

lats = df_train[['lat', 'lon']].drop_duplicates()['lat'].values
lons = df_train[['lat', 'lon']].drop_duplicates()['lon'].values

df_eval_summary = pd.DataFrame(columns=['output', 'rmse_cv', 'r2_cv', 'rmse_test', 'r2_test'])

In [ ]:
# sensitivity analysis setup
sims = ['RES', 'IND', 'TRA', 'AGR', 'ENE']
sens_inds_S1_ST = ['S1', 'S1_conf', 'ST', 'ST_conf']
ds_sens_ind = xr.Dataset({})

empty_values = np.empty((580, 1440))
empty_values[:] = np.nan
empty_da = xr.DataArray(empty_values, dims=('lat', 'lon'), coords={'lat': np.arange(-60, 85, 0.25), 'lon': np.arange(-180, 180, 0.25)})

for sim in sims:
    for sens_ind in sens_inds_S1_ST:
        ds_sens_ind.update({sens_ind + '_' + sim: empty_da})
        
sims_S2 = ['RES_IND', 'RES_TRA', 'RES_AGR', 'RES_ENE', 'IND_TRA', 'IND_AGR', 'IND_ENE', 'TRA_AGR', 'TRA_ENE', 'AGR_ENE']
sims_S2_indexes = [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
sens_inds_S2 = ['S2', 'S2_conf']

for sim in sims_S2:
    for sens_ind in sens_inds_S2:
        ds_sens_ind.update({sens_ind + '_' + sim: empty_da})

sens_inputs = {
    'num_vars': 5,
    'names': sims,
    'bounds': [[0.0, 1.5],
               [0.0, 1.5],
               [0.0, 1.5],
               [0.0, 1.5],
               [0.0, 1.5]]
}

sens_param_values = saltelli.sample(sens_inputs, 1000)

In [ ]:
def output_values(gridcell, df_train, df_test, output):
    """for a given gridcell, return the training and test data"""
    lat, lon = gridcell
    
    df_train_gridcell = df_train.loc[df_train.lat == lat].loc[df_train.lon == lon]
    df_test_gridcell = df_test.loc[df_test.lat == lat].loc[df_test.lon == lon]
    
    y_train, y_test = df_train_gridcell[output].values, df_test_gridcell[output].values
    
    return lat, lon, y_train, y_test

def create_emulator():
    """create a new gaussian process emulator"""
    emulator = make_pipeline(
        PowerTransformer(),
        GaussianProcessRegressor(
            kernel=Matern(length_scale=3.4000000000000004, nu=2.5), 
            n_restarts_optimizer=240, 
            normalize_y=False)
    )
    set_param_recursive(emulator.steps, 'random_state', 123)
    
    return emulator


def emulator_cv(emulator, X_train, y_train, y_test):
    """10-fold cross-validation on the emulator using the training data"""
    cv = cross_validate(emulator, X_train, y_train, cv=10, scoring={'r2': 'r2', 'rmse': 'neg_mean_squared_error'})
    
    return cv


def emulator_fit_save(emulator, X_train, y_train, path, output, lat, lon):
    """fit the emulator to the training data and save"""
    emulator.fit(X_train, y_train)
        
    joblib.dump(emulator, path + output + '/emulator_' + output + '_' + str(lat) + '_' + str(lon) + '.joblib')
    
    return emulator
    
    
def sensitivity_analysis(sens_inputs, sens_predictions):
    """determine the sensitivity indices of the emulator"""
    sens_ind_dict = sobol.analyze(sens_inputs, sens_predictions)
    
    return sens_ind_dict

In [ ]:
def emulator_with_sensitivity(gridcell):
    """run all the functions to create the emulator and run the sensitivity analysis"""
    lat, lon, y_train, y_test = output_values(gridcell, df_train, df_test, output)
    
    emulator = create_emulator()
    
    cv = emulator_cv(emulator, X_train, y_train, y_test)
    r2_cv = cv['test_r2']
    rmse_cv = np.sqrt(np.abs(cv['test_rmse']))
    
    emulator = emulator_fit_save(emulator, X_train, y_train, path, output, lat, lon)
    
    y_pred = emulator.predict(X_test)
    
    sens_predictions = emulator.predict(sens_param_values)
    sens_ind_dict = sensitivity_analysis(sens_inputs, sens_predictions)

    return lat, lon, output, y_test, y_pred, rmse_cv, r2_cv, sens_ind_dict

In [ ]:
output = 'PM2_5_DRY'
# ['PM2_5_DRY', 'o3', 'AOD550_sfc', 'asoaX_2p5', 'bc_2p5', 'bsoaX_2p5', 
# 'nh4_2p5', 'no3_2p5', 'oc_2p5', 'oin_2p5', 'so4_2p5']

bag_gridcells = db.from_sequence(df_train[['lat', 'lon']].drop_duplicates().values.tolist())
bag_gridcells = bag_gridcells.map(emulator_with_sensitivity)

In [ ]:
bag_gridcells.visualize()

In [ ]:
%%time
results = bag_gridcells.compute()

In [ ]:
y_test = np.array([result[3] for result in results]).ravel()
y_pred = np.array([result[4] for result in results]).ravel()
np.savez_compressed(path + output + '/y_test_pred_' + output + '.npz', y_test=y_test, y_pred=y_pred)

In [ ]:
rmse_cv = np.mean(np.array([result[5] for result in results]))
r2_cv = np.mean(np.array([result[6] for result in results]))
rmse_test = np.round(np.sqrt(np.abs(mean_squared_error(y_test, y_pred))), decimals=4)
r2_test = np.round(r2_score(y_test, y_pred), decimals=4)

df_eval_summary = df_eval_summary.append([{
        'output': output,
        'rmse_cv': rmse_cv, 
        'r2_cv': r2_cv,                                             
        'rmse_test': rmse_test, 
        'r2_test': r2_test}],              
        ignore_index=True, 
        sort=False)
df_eval_summary.to_csv(path + output + '/df_eval_summary_' + output + '.csv')
df_eval_summary

In [ ]:
lats = np.array([result[0] for result in results]).ravel()
lons = np.array([result[1] for result in results]).ravel()
sensitivities = [result[7] for result in results]

for index, sens in enumerate(sensitivities):
    lat = lats[index]
    lon = lons[index]

    for sim_index, sim in enumerate(sims):
        for sens_ind_index, sens_ind in enumerate(sens_inds_S1_ST):
            ds_sens_ind[sens_ind + '_' + sim] = xr.where(
                (ds_sens_ind.coords['lat'] == lat) & (ds_sens_ind.coords['lon'] == lon),
                sens[sens_ind][sim_index],
                ds_sens_ind[sens_ind + '_' + sim]
            )

    for sim_index, sim in enumerate(sims_S2):
        for sens_ind_index, sens_ind in enumerate(sens_inds_S2):
            ds_sens_ind[sens_ind + '_' + sim] = xr.where(
                (ds_sens_ind.coords['lat'] == lat) & (ds_sens_ind.coords['lon'] == lon),
                sens[sens_ind][sims_S2_indexes[sim_index][0], sims_S2_indexes[sim_index][1]],
                ds_sens_ind[sens_ind + '_' + sim]
            )

ds_sens_ind.to_netcdf(path + output + '/ds_sens_ind_' + output + '.nc')

In [ ]:
def make_plot(index, output, df_eval_summary, label, y_test, y_pred):
    ax = fig.add_subplot(gs[index])
    ax.set_facecolor('whitesmoke')
    limit = np.nanmax(y_pred)
    plt.xlim([0, limit])
    plt.ylim([0, limit])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('Simulator, ' + label, fontsize=14)
    plt.ylabel('Emulator, ' + label, fontsize=14)
    plt.scatter(np.vstack(y_test), np.vstack(y_pred))
    x = np.arange(2 * np.ceil(limit))
    plt.plot(x, x, '', color='grey', ls='--')
    plt.plot(x, 0.5 * x, '', color='grey', ls='--')
    plt.plot(x, 2 * x, '', color='grey', ls='--')
    text = "R$^2$ CV = " + str(np.round(np.nanmean(df_eval_summary['r2_cv'].values[0]), decimals=4)) + \
           "\nRMSE CV = " + str(np.round(np.nanmean(df_eval_summary['rmse_cv'].values[0]), decimals=4)) + \
           "\nR$^2$ test = " + str(np.round(np.nanmean(df_eval_summary['r2_test'].values[0]), decimals=4)) + \
           "\nRMSE test = " + str(np.round(np.nanmean(df_eval_summary['rmse_test'].values[0]), decimals=4))
    at = matplotlib.offsetbox.AnchoredText(text, prop=dict(size=11), frameon=True, loc='upper left')
    at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
    ax.add_artist(at)

In [ ]:
fig = plt.figure(1, figsize=(6, 6))
gs = gridspec.GridSpec(1, 1)

path = '/nobackup/earlacoa/machinelearning/data/'
outputs = ['PM2_5_DRY']#, 'o3']#, 'AOD550_sfc', 'asoaX_2p5']
labels = ['ambient PM$_{2.5}$ concentration\n(${\mu}g$ $m^{-3}$)',
          'ambient O$_{3}$ concentration\n($ppbv$)',
          'surface AOD 550 nm',
          'anthropogenic SOA$_{2.5}$ concentration\n(${\mu}g$ $m^{-3}$)']
pattern = r'([+-]?\d+.?\d+)'

for index, output in enumerate(outputs):
    y_values = np.load(path + output + '/y_test_pred_' + output + '.npz')
    y_test = y_values['y_test']
    y_pred = y_values['y_pred']
    df_eval_summary = pd.read_csv(path + output + '/df_eval_summary_' + output + '.csv')   
    make_plot(index, output, df_eval_summary, labels[index], y_test, y_pred)

gs.tight_layout(fig, rect=[0, 0, 1.0, 1.0])
plt.savefig(path + output + '/' + output + '_eval.png', dpi=700, alpha=True, bbox_inches='tight')
plt.savefig(path + output + '/' + output + '_eval.eps', format='eps', dpi=700, alpha=True, bbox_inches='tight')
plt.show()

In [ ]:
ds_sens_ind = xr.open_dataset(path + output + '/ds_sens_ind_' + output + '.nc')
lon = ds_sens_ind['S1_RES'].lon.values
lat = ds_sens_ind['S1_RES'].lat.values
xx, yy = np.meshgrid(lon, lat)

In [ ]:
fig = plt.figure(1, figsize=(12, 8))
gs = gridspec.GridSpec(2, 3)

sens = 'S1'
region = 'GBA'

label = 'ambient PM$_{2.5}$ concentrations'
#label = 'ambient O$_{3}$ concentrations'
#label = 'surface AOD 550 nm'
#label = 'anthropogenic SOA$_{2.5}$ concentrations'

sims = ['RES', 'IND', 'TRA', 'AGR', 'ENE']
levels = {}
levels.update({'China':(0,0.075,0.15,0.225,0.30,0.375,0.45,0.525,0.60,0.675,100000)})
levels.update({'GBA':(0,0.075,0.15,0.225,0.30,0.375,0.45,0.525,0.60,0.675,100000)})
cmap_colors = {}
cmap_colors.update({'S1': ['#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026']})
cb_label = {}
cb_label.update({'S1':'First-order sensitivity indices for ' + label})
plots = [(sim, sens, region, levels[region], cmap_colors[sens], cb_label[sens], label) for sim in sims]

for index, item in enumerate(plots):
    ax = fig.add_subplot(gs[index], projection=ccrs.PlateCarree())
    if item[2] == 'China':
        ax.set_extent([73, 135, 18, 54], crs=ccrs.PlateCarree())
        shape_feature = ShapelyFeature(Reader('/nobackup/earlacoa/health/data/china_taiwan_hongkong_macao.shp').geometries(),
                                       ccrs.PlateCarree(), facecolor='none')
        ax.patch.set_visible(False)
        ax.spines['geo'].set_visible(False)
        ax.add_feature(shape_feature, edgecolor='black', linewidth=0.5)
    elif item[2] == 'GBA':
        ax.set_extent([111.3, 115.5, 21.5, 24.5], crs=ccrs.PlateCarree())
        shape_feature1 = ShapelyFeature(Reader('/nobackup/earlacoa/health/data/gadm36_CHN_2.shp').geometries(),
                                        ccrs.PlateCarree(), facecolor='none')
        shape_feature2 = ShapelyFeature(Reader('/nobackup/earlacoa/health/data/gadm36_HKG_0.shp').geometries(),
                                        ccrs.PlateCarree(), facecolor='none')
        shape_feature3 = ShapelyFeature(Reader('/nobackup/earlacoa/health/data/gadm36_MAC_0.shp').geometries(),
                                        ccrs.PlateCarree(), facecolor='none')
        shape_feature4 = ShapelyFeature(Reader('/nobackup/earlacoa/health/data/gadm36_TWN_0.shp').geometries(),
                                        ccrs.PlateCarree(), facecolor='none')
        ax.add_feature(shape_feature1, edgecolor='black', linewidth=0.5)
        ax.add_feature(shape_feature2, edgecolor='black', linewidth=0.5)
        ax.add_feature(shape_feature3, edgecolor='black', linewidth=0.5)
        ax.add_feature(shape_feature4, edgecolor='black', linewidth=0.5)
    norm = matplotlib.colors.Normalize(vmin=item[3][0], vmax=item[3][-2])
    cmap = matplotlib.colors.ListedColormap(list(item[4]))
    im = ax.contourf(xx, yy, ds_sens_ind[item[1] + '_' + item[0]].values, item[3],
                     cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
    #plt.annotate(r'\textbf{' + chr(97 + index) + '}', xy=(0,1.05), xycoords='axes fraction', fontsize=14, weight='bold')
    plt.annotate(chr(97 + index), xy=(0,1.05), xycoords='axes fraction', fontsize=14, weight='bold')
    plt.title(item[0], size=14)

fig.canvas.draw()
gs.tight_layout(fig, rect=[0, 0, 0.95, 0.95], h_pad=1, w_pad=1) 

plt.draw()
    
ax_cbar = fig.add_axes([0.97, 0.15, 0.02, 0.7])
sm = plt.cm.ScalarMappable(
    norm=matplotlib.colors.Normalize(vmin=levels[region][0], vmax=levels[region][-2]),                       
    cmap=im.cmap
)
sm.set_array([])  
cb = plt.colorbar(
    sm, 
    cax=ax_cbar, 
    norm=matplotlib.colors.Normalize(vmin=levels[region][0], vmax=levels[region][-2]),              
    cmap=cmap_colors[sens], 
    ticks=levels[region][0:-1]
)
cb.set_label(cb_label[sens], size=14)
cb.ax.tick_params(labelsize=14)

plt.savefig(path + output + '/' + output + '_' + region + '_compare.png', dpi=700, alpha=True, bbox_inches='tight')
plt.savefig(path + output + '/' + output + '_' + region + '_compare.eps', format='eps', dpi=700, alpha=True, bbox_inches='tight')
plt.show()